<a href="https://www.kaggle.com/code/alihashemi1376/sentiment-analysis-llm-and-ssl?scriptVersionId=177558002" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/snappfood-persian-sentiment-analysis/Snappfood - Sentiment Analysis.csv
/kaggle/input/basalam-comments-and-products/BaSalam.reviews.csv
/kaggle/input/basalam-comments-and-products/BaSalam.products.csv


In [2]:
!pip install hazm  
!pip install transformers[torch]
!pip install -q clean-text[gpl]
!pip install hazm
!pip install sentence_transformers 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 892.6/892.6 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 67.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 44.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.4
    Uninstalling nltk-3.2.4:
      Successfully uninstalled nltk-3.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cubinlinker, which is not installed.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 re

In [3]:
import pandas as pd 
import numpy as np 
from sklearn.model_selection import train_test_split
from hazm import stopwords_list
from sentence_transformers import SentenceTransformer, models
from transformers import BertTokenizer
import pandas as pd
from torch.utils.data import Dataset,random_split,DataLoader
from torch.optim.lr_scheduler import ExponentialLR, CosineAnnealingWarmRestarts,PolynomialLR
from sklearn.model_selection import train_test_split
import re
import hazm
from cleantext import clean
import torch
import torch.nn as nn  



In [4]:
df = pd.read_csv('/kaggle/input/snappfood-persian-sentiment-analysis/Snappfood - Sentiment Analysis.csv',encoding='utf-8', sep='\t+', on_bad_lines='skip')


/tmp/ipykernel_24/2261564306.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('/kaggle/input/snappfood-persian-sentiment-analysis/Snappfood - Sentiment Analysis.csv',encoding='utf-8', sep='\t+', on_bad_lines='skip')


In [5]:
data_train , data_test = train_test_split(df,test_size=0.1, random_state=123)


In [6]:
!sudo pip install https://github.com/sobhe/hazm/archive/master.zip --upgrade   

     - 3.9 MB 594 kB/s
  Installing build dependencies ... - \ | done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.5 MB 3.8 MB/s 
     |████████████████████████████████| 1.1 MB 20.1 MB/s 
     |████████████████████████████████| 26.6 MB 24.5 MB/s 
     |████████████████████████████████| 11.1 MB 54.8 MB/s 
     |████████████████████████████████| 4.4 MB 24.7 MB/s 
     |████████████████████████████████| 17.3 MB 73.2 MB/s 
     |████████████████████████████████| 78 kB 6.1 MB/s 
     |████████████████████████████████| 301 kB 58.5 MB/s 
     |████████████████████████████████| 777 kB 67.3 MB/s 
     |████████████████████████████████| 97 kB 6.0 MB/s 
     |████████████████████████████████| 34.5 MB 48.8 MB/s 
     |████████████████████████████████| 61 kB 6.7 MB/s 
     |████████████████████████████████| 234 kB 64.9 MB/s 
     |████████████████████████████████| 83 kB 1.8 MB/s 
  Created wheel for hazm: filena

In [7]:
def remove_stopword(text):
    text = ' '.join([word for word in text.split() if word not in stopwords_list()])
    return text

def clean_unicode(text):
    wierd_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"U+23F0"
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u'\U00010000-\U0010ffff'
        u"\u200d"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\u3030"
        u"\ufe0f"
        u"\u2069"
        u"\u2066"
        u"\u200c"
        u"\u2068"
        u"\u2067"
        "]+", flags=re.UNICODE)

    text = wierd_pattern.sub(r'', text)    

    # removing extra spaces, hashtags
    text = re.sub("#", "", text)
    #text = re.sub("\s+", " ", text)
    return text

def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext


def clean_urls(text):
    text = text.strip() # remove leading an traling whitespace
    # regular cleaning
    text = clean(text,
        fix_unicode=True,
        to_ascii=False,
        lower=True,
        no_line_breaks=True,
        no_urls=True,
        no_emails=True,
        no_phone_numbers=True,
        no_numbers=False,
        no_digits=False,
        no_emoji=True,
        no_currency_symbols=True,
        no_punct=False,
        replace_with_url="",
        replace_with_email="",
        replace_with_phone_number="",
        replace_with_number="",
        replace_with_digit="0",
        replace_with_currency_symbol="",
    )
    return text



In [8]:
def preprocess(data):
    data['comment']=data['comment'].apply(remove_stopword)
    data['comment']=data['comment'].apply(cleanhtml)
    data['comment']=data['comment'].apply(clean_unicode)
    data['comment']=data['comment'].apply(clean_urls)
    # Load pre-trained BERT tokenizer
    #tokenizer = BertTokenizer.from_pretrained('HooshvareLab/bert-base-parsbert-uncased')
    #data_desc = data['Text'].tolist()
    # Tokenize the data
    #tokenized_data = [tokenizer.encode(text, add_special_tokens=True) for text in data_desc]
    # Convert token IDs to strings
    #tokenized_data_str = [' '.join(tokenizer.convert_ids_to_tokens(ids)) for ids in tokenized_data]
    # Load SentenceTransformer model
    model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
    data_desc=data['comment'].tolist()
    data_embeddings = model.encode(data_desc)
    # Encode the tokenized data using SentenceTransformer
    #data_embeddings = model.encode(tokenized_data_str)
    return data_embeddings 



In [9]:
embedded_train=preprocess(data_train)        
embedd_test=preprocess(data_test)      

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.13k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/723 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1969 [00:00<?, ?it/s]

Batches:   0%|          | 0/219 [00:00<?, ?it/s]

In [10]:
class MyClassifier(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        l1 = nn.Linear(input_dim, 512)
        b1 = nn.BatchNorm1d(512)
        r1 = nn.LeakyReLU()
        d1 = nn.Dropout(p = 0.2)
        
        l2 = nn.Linear(512, 128)
        b2 = nn.BatchNorm1d(128)
        r2 = nn.LeakyReLU()
        d2 = nn.Dropout(p = 0.2)
        
        l3 = nn.Linear(128, 64)
        b3 = nn.BatchNorm1d(64)
        r3 = nn.LeakyReLU()
        d3 = nn.Dropout(p = 0.2)
        
        l4 = nn.Linear(64,32)
        b4 = nn.BatchNorm1d(32)
        r4 = nn.LeakyReLU()
        d4 = nn.Dropout(p = 0.2)
        
        
        self.seq = nn.Sequential(l1,b1 ,r1, d1,l2,b2 ,r2, d2,l3,b3 ,r3, d3,l4,b4 ,r4, d4)
        self.fc1 = nn.Linear(32, num_classes)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        h = self.seq(x)
        f = self.fc1(h)
        return self.softmax(f)
    
        

In [11]:
class CustomDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        feature = self.features[idx]
        label = self.labels[idx]
        return feature, label
    
custom_train = CustomDataset(embedded_train, torch.Tensor(data_train.label_id).long())
custom_test = CustomDataset(embedd_test, torch.Tensor(data_test.label_id.values).long())

In [12]:
train_loader = torch.utils.data.DataLoader(custom_train, batch_size=256, shuffle=True)
valid_loader = torch.utils.data.DataLoader(custom_test, batch_size=256, shuffle=True)

In [13]:
def fit(clf,
        train_loader,
        optimizer,
        criterian,
        scheduler):

    clf.train()
    training_loss_running = 0
    training_correct_running = 0
    total = 0
    counter = 0
    for i,data in enumerate(train_loader):
        counter += 1
        data,label = data[0],data[1]
        #data = data.reshape(-1,28*28)
        total += label.size(0)
        optimizer.zero_grad()            
        out = clf(data)            
        loss = criterian(out, label)                
        training_loss_running += loss.item()
        _,pred = torch.max(out.data,1)
        training_correct_running += (pred == label).sum().item()
        loss.backward()
        optimizer.step()
    scheduler.step() 
    train_loss = training_loss_running / counter
    train_accuracy = 100. * training_correct_running / total
    print(optimizer.param_groups[0]['lr'])
    return train_loss, train_accuracy     



In [14]:
def validation (clf,validation_loader,criterian,epoch):
    clf.eval()
    valid_loss_running = 0
    valid_acc_running = 0
    total = 0
    counter = 0
    for i,data in enumerate(validation_loader):
        counter += 1
        data,label = data[0],data[1]
        #data = data.reshape(-1,28*28)
        total += label.size(0)
        out = clf(data)
        loss = criterian(out, label)
        valid_loss_running += loss.item()
        _,pred = torch.max(out.data,1)
        valid_acc_running += (pred == label).sum().item()

    valid_loss = valid_loss_running / counter
    valid_acc = 100. * valid_acc_running / total  
      

    return valid_loss,valid_acc
       


In [15]:
def train (hyparam,train_loader,val_loader):

    clf = MyClassifier(768,2)
    #optimizer = torch.optim.Adam(clf.parameters(), lr =hyparam['lr'])
    optimizer = torch.optim.Adam(clf.parameters(), lr=hyparam['lr'],weight_decay=0.001)
    criterian = nn.CrossEntropyLoss()
    scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=50, T_mult=1)
    #scheduler = PolynomialLR(optimizer,  total_iters=5, power=1)
    train_loss =[]
    train_acc =[]
    val_loss =[]
    val_acc = []
    
    for epoch in range(hyparam['epoch']):
        print(f"Epoch {epoch+1} of {hyparam['epoch']}")
        training_loss,training_acc = fit(clf,train_loader,optimizer,criterian,scheduler)
        validation_loss,validation_acc = validation(clf,val_loader,criterian,epoch)

        train_loss.append(training_loss)
        train_acc.append(training_acc)

        val_loss.append(validation_loss)
        val_acc.append(validation_acc)

        

       
        
    
        print(f"Train Loss: {training_loss:.4f}, Train Acc: {training_acc:.2f},\
         Val Loss: {validation_loss:.4f}, Val Acc: {validation_acc:.2f}")



      
    return clf,train_loss,train_acc,val_loss,val_acc
        



In [16]:
hparams = {'batch_size': 256, 'lr': 0.4e-4, 'epoch': 20} #6e-4      
clf,train_loss,train_acc,val_loss,val_acc = train(hparams,train_loader,valid_loader) 


Epoch 1 of 20
3.9960534568565436e-05
Train Loss: 0.5993, Train Acc: 71.61,         Val Loss: 0.5560, Val Acc: 77.64
Epoch 2 of 20
3.9842294026289565e-05
Train Loss: 0.5500, Train Acc: 77.57,         Val Loss: 0.5343, Val Acc: 78.73
Epoch 3 of 20
3.964574501457378e-05
Train Loss: 0.5326, Train Acc: 78.71,         Val Loss: 0.5230, Val Acc: 79.66
Epoch 4 of 20
3.9371663222572625e-05
Train Loss: 0.5221, Train Acc: 79.41,         Val Loss: 0.5135, Val Acc: 80.13
Epoch 5 of 20
3.9021130325903076e-05
Train Loss: 0.5144, Train Acc: 80.06,         Val Loss: 0.5086, Val Acc: 80.21
Epoch 6 of 20
3.859552971776503e-05
Train Loss: 0.5091, Train Acc: 80.31,         Val Loss: 0.5054, Val Acc: 80.67
Epoch 7 of 20
3.809654104932039e-05
Train Loss: 0.5047, Train Acc: 80.81,         Val Loss: 0.5017, Val Acc: 80.77
Epoch 8 of 20
3.7526133600877275e-05
Train Loss: 0.5010, Train Acc: 81.03,         Val Loss: 0.5005, Val Acc: 80.99
Epoch 9 of 20
3.6886558510040305e-05
Train Loss: 0.4976, Train Acc: 81.35, 

In [17]:
basalam = pd.read_csv('/kaggle/input/basalam-comments-and-products/BaSalam.reviews.csv')
basalam = basalam.description.dropna() 

/tmp/ipykernel_24/2086414997.py:1: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.
  basalam = pd.read_csv('/kaggle/input/basalam-comments-and-products/BaSalam.reviews.csv')


In [18]:
basalam = basalam[0:10000]
df = pd.DataFrame([])
df['comment'] = basalam 
embdd = preprocess(df)
df_embdd_ublabeled = pd.DataFrame(embdd) 
df_embdd_labeled = pd.DataFrame(embedded_train) 




Batches:   0%|          | 0/313 [00:00<?, ?it/s]

In [19]:
i = 0
X_train = df_embdd_labeled.copy()
y_train = data_train.label_id.copy()
while (i<6):
    
    print('start iteration', i, 'unlabeled-shape', df_embdd_ublabeled.shape)
    df_embdd_ublabeled_tensor = torch.Tensor(df_embdd_ublabeled.values)
    pred_probs = clf(df_embdd_ublabeled_tensor)
    _,preds = torch.max(pred_probs.data,1)
    prob_0 = pred_probs[:,0]
    prob_1 = pred_probs[:,1]
    df_pred_prob = pd.DataFrame([])
    df_pred_prob['preds'] = preds
    df_pred_prob['prob_0'] = prob_0.detach().numpy()
    df_pred_prob['prob_1'] = prob_1.detach().numpy()
    df_pred_prob.index = df_embdd_ublabeled.index

    train_f1s = []
    test_f1s = []
    pseudo_labels = []


    high_prob = [1] 


    high_prob = pd.concat([df_pred_prob.loc[df_pred_prob['prob_0'] > 0.99],
                               df_pred_prob.loc[df_pred_prob['prob_1'] > 0.99]],
                              axis=0)
    
    print('high-prob',high_prob.shape) 
    X_train = pd.concat([X_train, df_embdd_ublabeled.loc[high_prob.index]], axis=0) 
    y_train = pd.concat([y_train, high_prob.preds]) 
    print ('x-train',X_train.shape)
    df_embdd_ublabeled = df_embdd_ublabeled.drop(index=high_prob.index)



    custom_data = CustomDataset(X_train.values,torch.Tensor(y_train.values).long())
    train_loader = torch.utils.data.DataLoader(custom_data, batch_size=256, shuffle=True)

    hparams ={'batch_size': 256, 'lr': 0.4e-4, 'epoch': 20} #6e-4      
    clf,train_loss,train_acc,val_loss,val_acc = train(hparams,train_loader,valid_loader) 
    
    i = i + 1
    print('end iteration', i ) 
    

start iteration 0 unlabeled-shape (10000, 768)
high-prob (6583, 3)
x-train (69583, 768)
Epoch 1 of 20
3.9960534568565436e-05
Train Loss: 0.5757, Train Acc: 76.04,         Val Loss: 0.5531, Val Acc: 77.74
Epoch 2 of 20
3.9842294026289565e-05
Train Loss: 0.5278, Train Acc: 80.04,         Val Loss: 0.5285, Val Acc: 79.14
Epoch 3 of 20
3.964574501457378e-05
Train Loss: 0.5102, Train Acc: 81.10,         Val Loss: 0.5182, Val Acc: 79.77
Epoch 4 of 20
3.9371663222572625e-05
Train Loss: 0.5005, Train Acc: 81.64,         Val Loss: 0.5103, Val Acc: 79.93
Epoch 5 of 20
3.9021130325903076e-05
Train Loss: 0.4937, Train Acc: 82.03,         Val Loss: 0.5086, Val Acc: 80.30
Epoch 6 of 20
3.859552971776503e-05
Train Loss: 0.4887, Train Acc: 82.42,         Val Loss: 0.5035, Val Acc: 80.34
Epoch 7 of 20
3.809654104932039e-05
Train Loss: 0.4851, Train Acc: 82.72,         Val Loss: 0.5017, Val Acc: 80.71
Epoch 8 of 20
3.7526133600877275e-05
Train Loss: 0.4811, Train Acc: 82.99,         Val Loss: 0.5000, Va

In [20]:
250 + 70906

71156